In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv('df_cars_ok.csv', sep='\t')
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[0], axis=1)
df.head(3)

,arquivo,nome,ano,km,cor,cambio,portas,preco
0,./dados/dados_carro\Carro-100.html,Chevrolet Onix 1.0 LT (Flex) 2020,2020 / 2020,0.000,Laranja,manual,3.0,"59.890,00"
1,./dados/dados_carro\Carro-1004.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,41.070,Branco,manual,2.0,"45.900,00"
2,./dados/dados_carro\Carro-1005.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,48.654,Prata,manual,4.0,"45.900,00"


In [4]:
for x in range(100):
    if x >= 10:
        str_replace = ',' + str(x)
    else:    
        str_replace = ',0' + str(x)
    df['preco'] = df.preco.replace({str_replace:''}, regex=True)

In [5]:
df['preco'].astype(float)

0       59.89
1       45.90
2       45.90
3       45.99
4       45.99
        ...  
1706    45.59
1707    45.73
1708    45.89
1709    45.89
1710    45.89
Name: preco, Length: 1711, dtype: float64

In [6]:
df.to_csv('teste.csv', sep='\t', encoding='utf-8')

In [7]:
df222 = pd.read_csv('teste.csv', sep='\t')

In [8]:
df222

,Unnamed: 0,arquivo,nome,ano,km,cor,cambio,portas,preco
0,0,./dados/dados_carro\Carro-100.html,Chevrolet Onix 1.0 LT (Flex) 2020,2020 / 2020,0.000,Laranja,manual,3.0,59.89
1,1,./dados/dados_carro\Carro-1004.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,41.070,Branco,manual,2.0,45.90
2,2,./dados/dados_carro\Carro-1005.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,48.654,Prata,manual,4.0,45.90
3,3,./dados/dados_carro\Carro-1008.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,19.000,Prata,manual,2.0,45.99
4,4,./dados/dados_carro\Carro-1009.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,24.715,Branco,manual,2.0,45.99
...,...,...,...,...,...,...,...,...,...
1706,1706,./dados/dados_carro\Carro-995.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,40.912,Branco,manual,2.0,45.59
1707,1707,./dados/dados_carro\Carro-996.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Du...,2019 / 2019,35.271,Branco,manual,2.0,45.73
1708,1708,./dados/dados_carro\Carro-997.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,40.741,Prata,manual,2.0,45.89
1709,1709,./dados/dados_carro\Carro-998.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Du...,2019 / 2019,40.870,Prata,manual,2.0,45.89


In [9]:
df

,arquivo,nome,ano,km,cor,cambio,portas,preco
0,./dados/dados_carro\Carro-100.html,Chevrolet Onix 1.0 LT (Flex) 2020,2020 / 2020,0.000,Laranja,manual,3.0,59.890
1,./dados/dados_carro\Carro-1004.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,41.070,Branco,manual,2.0,45.900
2,./dados/dados_carro\Carro-1005.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,48.654,Prata,manual,4.0,45.900
3,./dados/dados_carro\Carro-1008.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,19.000,Prata,manual,2.0,45.990
4,./dados/dados_carro\Carro-1009.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,24.715,Branco,manual,2.0,45.990
...,...,...,...,...,...,...,...,...
1706,./dados/dados_carro\Carro-995.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,40.912,Branco,manual,2.0,45.590
1707,./dados/dados_carro\Carro-996.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Du...,2019 / 2019,35.271,Branco,manual,2.0,45.730
1708,./dados/dados_carro\Carro-997.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,40.741,Prata,manual,2.0,45.890
1709,./dados/dados_carro\Carro-998.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Du...,2019 / 2019,40.870,Prata,manual,2.0,45.890


In [10]:
#df_km = df[df['km'] == 0.0]
#df_km.shape

In [11]:
#df.loc[df['km'] == 0.0, 1] = 1
#df_km = df[df['km'] == 0.0] 
#df_km.shape

### Criando a baseline

In [12]:
df.describe()

,km,portas
count,1711.000000,1486.000000
mean,30.362317,3.983176
std,75.420014,0.837898
min,0.000000,2.000000
25%,0.000000,4.000000
50%,13.000000,4.000000
75%,40.692000,4.000000
max,999.999000,5.000000


In [13]:
y = df.preco

In [14]:
y.head(3)

0    59.890
1    45.900
2    45.900
Name: preco, dtype: object

In [15]:
cars_features = ['km', 'portas']
X = df[cars_features]

In [16]:
X.head(3)

,km,portas
0,0.000,3.0
1,41.070,2.0
2,48.654,4.0


In [17]:
#s = (X.dtypes == 'object')
#object_cols = list(s[s].index)

In [18]:
#type(X['km'][0])

In [19]:
#df_km = X_train[X_train['km'] == 0.0] 
#df_km.shape

In [20]:
#training_y = y_train.dropna()

In [21]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.4, random_state=42)

In [48]:
train_X.fillna(train_X.mean(), inplace=True)
val_X.fillna(val_X.mean(), inplace=True)
train_y.fillna(train_X.mean(), inplace=True)
val_y.fillna(train_X.mean(), inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [66]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', multi_class='ovr')
lr.fit(train_X, train_y)
lr.score(val_X, val_y)

0.035036496350364967

In [68]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)
rf.fit(train_X, train_y)
rf.score(val_X, val_y)

0.033576642335766425

In [69]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(train_X, train_y)
lr.score(val_X, val_y)

0.035036496350364967

In [86]:
#model = DecisionTreeRegressor(random_state=1)
model = DecisionTreeRegressor(max_leaf_nodes=20, random_state=1)
model.fit(train_X, train_y)
predicted_model = model.predict(val_X)
mean_absolute_error(val_y, predicted_model)

12.215349057610965

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


X_train.fillna(train_X.mean(), inplace=True)
X_test.fillna(val_X.mean(), inplace=True)
y_train.fillna(train_X.mean(), inplace=True)
y_test.fillna(train_X.mean(), inplace=True)


dt = DecisionTreeRegressor(random_state=0, criterion="mae")
dt_fit = dt.fit(X_train, y_train)

dt_scores = cross_val_score(dt_fit, X_train, y_train, cv = 15)
print("mean cross validation score: {}".format(np.mean(dt_scores)))
print("score without cv: {}".format(dt_fit.score(X_train, y_train)))

# on the test or hold-out set
from sklearn.metrics import r2_score
print(r2_score(y_test, dt_fit.predict(X_test)))
print(dt_fit.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


mean cross validation score: -0.09866040727375879
score without cv: 0.32257681834400176
-0.08639675698768068
-0.08639675698768068


12.215349057610965

In [51]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [56]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: ", max_leaf_nodes , "  \t\t Mean Absolute Error:", my_mae)

Max leaf nodes:  5   		 Mean Absolute Error: 12.20638117421985
Max leaf nodes:  50   		 Mean Absolute Error: 12.332250330296228
Max leaf nodes:  500   		 Mean Absolute Error: 13.062250847023073
Max leaf nodes:  5000   		 Mean Absolute Error: 13.062250847023073


In [ ]:
for x in range(100):
    print(predicted_model[x])

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

In [79]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

In [80]:
X_train.fillna(train_X.mean(), inplace=True)
X_test.fillna(val_X.mean(), inplace=True)
y_train.fillna(train_X.mean(), inplace=True)
y_test.fillna(train_X.mean(), inplace=True)


In [81]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.042801556420233464

In [82]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.05642023346303502

In [83]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.038910505836575876

In [84]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [85]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [88]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [90]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(X, y):
    X_train, X_test, y_train, y_test = X[train_index], y[test_index], \
                                       X[train_index], y[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


KeyError: "None of [Int64Index([   6,   10,   13,   18,   19,   22,   25,   30,   40,   43,\n            ...\n            1700, 1701, 1702, 1703, 1704, 1705, 1706, 1708, 1709, 1710],\n           dtype='int64', length=1140)] are in the [columns]"